# Acquire ACS Data Over Serial WITHOUT Knowledge From the Device File
For this example, you will need to connect an ACS to your computer via serial. 
The sensor should be on before running this notebook. Note that the ACS is in picoDOS for the first 10 seconds after it is powered on and may not register on a serial port.

Note: In this example, absorption and attenuation will look bad because the ACS is running in air.


In [1]:
from datetime import datetime, timezone

from acspype import ACSStream
from acspype.processing import convert_sn_str
import acspype.qaqc as acsqaqc
from acspype.utils import find_acs_port # This is a convenience function that finds the first port that has an ACS attached to it.

In [2]:
num_loops = 5  # Acquire X packets, and then stop. Otherwise, the while loop will run forever.

In [3]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
with ACSStream(port) as acss: # Open a serial object.
    i = 0
    while True:  # Continuously read the stream until a full packet is found.
        acss.read_stream()
        acs_pkt = acss.find_packet()
        if acs_pkt.full_packet is not None:
            parsed_packet = acss.parse_packet(acs_packet = acs_pkt) # Parse the packet.
            
            # Run the gap and syntax tests.
            flag_gap = acsqaqc.gap_test(now = datetime.now(timezone.utc), 
                                        time_stmp = acs_pkt.daq_time, 
                                        buffer_length= len(acss._buffer), # The bytes in the serial buffer can be accessed with the _buffer attribute. This is really the only instance where you would need access to the buffer contents outside of the ACSStream class.
                                        record_length = parsed_packet.record_length)
            flag_syntax = acsqaqc.syntax_test(full_packet = acs_pkt.full_packet)
            
            # Print some of the results.
            print(f"---------- Packet {i} ----------")
            print(f"Acquisition Time: {parsed_packet.daq_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'}")
            print(f"Acquired Packet From: {convert_sn_str(parsed_packet.sn_int)}")
            print(f"Elapsed Time (ms): {parsed_packet.elapsed_time}")
            print(f"Absorption Signal Counts: {parsed_packet.a_signal}")
            if flag_gap == 1:
                print('Gap Test: Passed')
            elif flag_gap == 4:
                print('Gap Test: Failed')
            if flag_syntax == 1:
                print('Syntax Test: Passed')
            elif flag_syntax == 4:
                print('Syntax Test: Failed')
            print('\n')
            
            i += 1
            if i == num_loops:
                break

---------- Packet 0 ----------
Acquisition Time: 2025-04-30T20:34:01.402Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 925376
Absorption Signal Counts: (436, 503, 588, 690, 813, 941, 1083, 1260, 1453, 1685, 1933, 2171, 2433, 2724, 3055, 3396, 3777, 4171, 4613, 5120, 5655, 6188, 6795, 7394, 8003, 8627, 9212, 9879, 10621, 11321, 12032, 12854, 13707, 14729, 15784, 16783, 17814, 18766, 19828, 20844, 21674, 22608, 23493, 24436, 25548, 26312, 28266, 29122, 29955, 30660, 31354, 31959, 32618, 33257, 33582, 33760, 33882, 33919, 33849, 33699, 33281, 32617, 31863, 31001, 30595, 30268, 29373, 28412, 27454, 26360, 25312, 24258, 23073, 21704, 20361, 19245, 18184, 17027, 15872, 14773, 13697, 12731, 11827, 10910)
Gap Test: Passed
Syntax Test: Passed


---------- Packet 1 ----------
Acquisition Time: 2025-04-30T20:34:01.660Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 925627
Absorption Signal Counts: (437, 505, 589, 690, 812, 943, 1085, 1259, 1454, 1684, 1932, 2172, 2432, 2724, 3056, 3396,